In [83]:
"""resize png to be smaller by a factor of 4 to fit in GPU"""
# from PIL import Image
# import os

# path = "radpng"
# resize_ratio = 0.25  # where 0.5 is half size, 2 is double size

# def resize_aspect_fit():
#     dirs = os.listdir(path)
#     for item in dirs:
#         image = Image.open(path+item)
#         file_path, extension = os.path.splitext(path+item)

#         new_image_height = int(image.size[0] / (1/resize_ratio))
#         new_image_length = int(image.size[1] / (1/resize_ratio))

#         image = image.resize((new_image_height, new_image_length), Image.ANTIALIAS)
#         image.save(file_path + "_small" + extension, 'PNG', quality=90)


# resize_aspect_fit()

'resize png to be smaller by a factor of 4 to fit in GPU'

In [97]:
import torch
import random
from PIL import Image
import numpy as np
import os
# Constants
NUM_INDIVIDUALS = 1
NUM_GENERATIONS = 500
NUM_RADICALS = 1

radicals_directory = "radpng210"
radical_files = [f for f in os.listdir(radicals_directory) if f.endswith('.png')]
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Load the target line art and kanji radicals
target = Image.open("myart.jpg").convert('L')
target_tensor = torch.tensor(np.array(target, dtype=np.float32)).to(device)
kanji_radicals = [torch.tensor(np.array(Image.open(os.path.join(radicals_directory, f)).convert('L'), dtype=np.float32)).to(device) for f in radical_files]

max_x = target.width - 400
max_y = target.height - 400

# Initialization
def create_individual():
    individual = []
    for _ in range(NUM_RADICALS):
        radical = random.choice(kanji_radicals)
        x = random.randint(0, max_x)
        y = random.randint(0, max_y)
        size = random.uniform(0.5, 1.5)
        individual.append((radical, x, y, size))
    return individual


population = [create_individual() for _ in range(NUM_INDIVIDUALS)]

# Fitness function
def fitness(individual):
    canvas = torch.ones_like(target_tensor) * 255
    for radical, x, y, size in individual:
        scaled_radical = torch.nn.functional.interpolate(radical.unsqueeze(0).unsqueeze(0), scale_factor=size, mode='bilinear').squeeze()

        # Clamp positions and sizes to ensure they don't go out of bounds
        y_end = min(y + scaled_radical.shape[0], canvas.shape[0])
        x_end = min(x + scaled_radical.shape[1], canvas.shape[1])
        
        scaled_radical = scaled_radical[:y_end-y, :x_end-x]

        canvas[y:y_end, x:x_end] = canvas[y:y_end, x:x_end] * (1 - scaled_radical/255.0)



    error = torch.sum((canvas - target_tensor)**2)
    return -error.item()


# Selection, Crossover, and Mutation
def crossover(parent1, parent2):
    split = random.randint(0, NUM_RADICALS-1)
    child = parent1[:split] + parent2[split:]
    return child

def mutate(individual):
    idx = random.randint(0, NUM_RADICALS-1)
    individual[idx] = (random.choice(kanji_radicals), random.randint(0, target.width), random.randint(0, target.height), random.uniform(0.5, 1.5))
    return individual

# Evolution
for generation in range(NUM_GENERATIONS):
    if generation % 50 == 0:
        print(f"Processing generation {generation}...")
    population.sort(key=fitness, reverse=True)
    new_population = population[:10]
    while len(new_population) < NUM_INDIVIDUALS:
        parent1 = random.choice(population[:10])
        parent2 = random.choice(population[:10])
        child = crossover(parent1, parent2)
        if random.random() < 0.1:
            child = mutate(child)
        new_population.append(child)
    population = new_population

# Display the best individual
best = population[0]
canvas = torch.ones_like(target_tensor) * 255
for radical, x, y, size in best:
    scaled_radical = torch.nn.functional.interpolate(radical.unsqueeze(0).unsqueeze(0), scale_factor=size, mode='bilinear').squeeze()

    # Clamp positions and sizes to ensure they don't go out of bounds
    y_end = min(y + scaled_radical.shape[0], canvas.shape[0])
    x_end = min(x + scaled_radical.shape[1], canvas.shape[1])
    
    scaled_radical = scaled_radical[:y_end-y, :x_end-x]

    canvas[y:y_end, x:x_end] -= scaled_radical

Image.fromarray(canvas.cpu().numpy().astype(np.uint8)).show()
output_path = "output_image.png"
Image.fromarray(canvas.cpu().numpy().astype(np.uint8)).save(output_path)
print(f"Image saved to {output_path}")


Processing generation 0...
Processing generation 50...
Processing generation 100...
Processing generation 150...
Processing generation 200...
Processing generation 250...
Processing generation 300...
Processing generation 350...
Processing generation 400...
Processing generation 450...
Image saved to output_image.png


In [115]:
def load_radical_as_tensor(image_path):
    img = Image.open(image_path)
    
    # If image has an alpha channel (transparency)
    if img.mode == 'RGBA':
        r, g, b, a = img.split()
        bg = Image.new('RGB', img.size, (255, 255, 255))  # Pure white background
        bg.paste(img, (0, 0), a)  # Paste image over white background where it's not transparent
        img = bg
        
    return torch.tensor(np.array(img.convert('L'), dtype=np.float32)).to(device)

kanji_radicals = [load_radical_as_tensor(os.path.join(radicals_directory, f)) for f in radical_files]

In [117]:
import torch
import random
from PIL import Image
import numpy as np
import os

# Constants
NUM_INDIVIDUALS = len(radical_files)  # Set the number of individuals equal to the number of radicals
NUM_GENERATIONS = 500
NUM_RADICALS = 1  # Each individual will now represent just one radical

radicals_directory = "radpng"
radical_files = [f for f in os.listdir(radicals_directory) if f.endswith('.png')]

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the target line art and kanji radicals
target = Image.open("myart.jpg").convert('L')
target_tensor = torch.tensor(np.array(target, dtype=np.float32)).to(device)
# kanji_radicals = [torch.tensor(np.array(Image.open(os.path.join(radicals_directory, f)).convert('L'), dtype=np.float32)).to(device) for f in radical_files]

max_x = target.width - 400
max_y = target.height - 400

# Initialization
def create_individual():
    radical = random.choice(kanji_radicals)
    x = random.randint(0, max_x)
    y = random.randint(0, max_y)
    size = random.uniform(0.5, 1.5)
    return [(radical, x, y, size)]

population = [create_individual() for _ in range(NUM_INDIVIDUALS)]
# Fitness function
def fitness(individual):
    canvas = torch.ones_like(target_tensor) * 255
    for radical, x, y, size in individual:
        scaled_radical = torch.nn.functional.interpolate(radical.unsqueeze(0).unsqueeze(0), scale_factor=size, mode='bilinear').squeeze()

        # Clamp positions and sizes to ensure they don't go out of bounds
        y_end = min(y + scaled_radical.shape[0], canvas.shape[0])
        x_end = min(x + scaled_radical.shape[1], canvas.shape[1])
        
        scaled_radical = scaled_radical[:y_end-y, :x_end-x]

        canvas[y:y_end, x:x_end] = canvas[y:y_end, x:x_end] * (scaled_radical/255.0)



    error = torch.sum((canvas - target_tensor)**2)
    return -error.item()


# Selection, Crossover, and Mutation
def crossover(parent1, parent2):
    split = random.randint(0, NUM_RADICALS-1)
    child = parent1[:split] + parent2[split:]
    return child


def mutate(individual):
    individual[0] = (random.choice(kanji_radicals), random.randint(0, target.width), random.randint(0, target.height), random.uniform(0.5, 1.5))
    return individual

for generation in range(NUM_GENERATIONS):
    if generation % 50 == 0:
        print(f"Processing generation {generation}...")
    population.sort(key=fitness, reverse=True)
    new_population = population[:10]
    while len(new_population) < NUM_INDIVIDUALS:
        parent1 = random.choice(population[:10])
        parent2 = random.choice(population[:10])
        child = crossover(parent1, parent2)
        if random.random() < 0.1:
            child = mutate(child)
        new_population.append(child)
    population = new_population

# Display the best individual
best = population[0]
canvas = torch.ones_like(target_tensor) * 255
for radical, x, y, size in best:
    scaled_radical = torch.nn.functional.interpolate(radical.unsqueeze(0).unsqueeze(0), scale_factor=size, mode='bilinear').squeeze()

    # Clamp positions and sizes to ensure they don't go out of bounds
    y_end = min(y + scaled_radical.shape[0], canvas.shape[0])
    x_end = min(x + scaled_radical.shape[1], canvas.shape[1])
    
    scaled_radical = scaled_radical[:y_end-y, :x_end-x]

    canvas[y:y_end, x:x_end] = canvas[y:y_end, x:x_end] * (scaled_radical/255.0)

Image.fromarray(canvas.cpu().numpy().astype(np.uint8)).show()
output_path = "output_image.png"
# Image.fromarray(canvas.cpu().numpy().astype(np.uint8)).save(output_path)
# print(f"Image saved to {output_path}")


Processing generation 0...
Processing generation 50...
Processing generation 100...
Processing generation 150...
Processing generation 200...
Processing generation 250...
Processing generation 300...
Processing generation 350...
Processing generation 400...
Processing generation 450...
